In [ ]:
import json
import os
import pandas as pd
import requests
import threading
import time
from dotenv import load_dotenv
from mixpanel import Mixpanel

# ------------
# INSTRUCTIONS
#
# Set the from_date and to_date variables to define the frame of data you want to load.
# Then, run this cell. The first time you run it, it will get all the Mixpanel events
# in the time range, then it will save that data to a json file in /app/analytics/.
# The next time you run this cell, it will load from the json file (unless you change the dates).
#
# ------------
# Helpful Dates
#
# Completed pilots:
#   LA LWC May run:           2025-05-18 - 2025-06-30
#   AZ Constrained MAC Pilot: 2025-06-13 - 2025-08-13 7:29pm MST
# Incomplete pilots:
#   LA LWC August Run:        2025-08-17            - ??? Final day to query before publish date of report
#   AZ Expanded MAC Pilot:    2025-08-13 7:30pm MST - ??? Final day to query before publish date of report
#
# -------------

from_date = '2025-05-18'
to_date = '2025-06-30'
client_agency = 'la_ldh'

# Credentials
project_root = os.path.dirname(os.getcwd())
dotenv_path = os.path.join(project_root, '.env.local')
load_dotenv(dotenv_path=dotenv_path)
SA_USERNAME = os.getenv("MIXPANEL_SERVICE_ACCOUNT_USERNAME").strip()
SA_SECRET = os.getenv("MIXPANEL_SERVICE_ACCOUNT_SECRET").strip()
PROJECT_ID = os.getenv("MIXPANEL_PROJECT_ID").strip()

# Endpoint
API_ENDPOINT = 'https://data.mixpanel.com/api/2.0/export'
headers = {"accept": "text/plain"}

# Date range
file_name = f"mixpanel_data_{from_date}_to_{to_date}.json"
params = {
    'from_date': from_date,
    'to_date': to_date,
    'project_id': PROJECT_ID
}

# Loading animation setup
is_loading = False

def spinning_cursor():
    while is_loading:
        for cursor in '|/-\\':
            print(f"\r{cursor}", end="", flush=True)
            time.sleep(0.1)
    # Clear the spinner line after loading is complete
    print("\r" + " " * 20 + "\r", end="", flush=True)

# Check if the data has already been downloaded
if os.path.exists(file_name):
    print(f"Loading data from local file: {file_name}")
    is_loading = True
    spinner_thread = threading.Thread(target=spinning_cursor)
    spinner_thread.start()
    
    with open(file_name, 'r') as f:
        raw_response_text = f.read()
    
    is_loading = False
    spinner_thread.join()
else:
    is_loading = True
    spinner_thread = threading.Thread(target=spinning_cursor)
    spinner_thread.start()
    
    try:
        # Make the request
        print("Fetching data from Mixpanel...")
        response = requests.get(
            API_ENDPOINT,
            headers=headers,
            params=params,
            auth=(SA_USERNAME, SA_SECRET)
        )
        
        is_loading = False
        spinner_thread.join()
        
        if response.status_code == 200:
            print("Successfully fetched data from API.")
            raw_response_text = response.text
            # Save the raw response to the file for future use
            with open(file_name, 'w') as f:
                f.write(raw_response_text)
            print(f"Data saved to {file_name}")
        else:
            print(f"Error: API request failed with status code {response.status_code}")
            print(response.text)
            exit()
    except:
        is_loading = False
        spinner_thread.join()
        print(f"\nAn error occurred during the API request: {e}")
        exit()

# Process the data (either from the file or the fresh API call)
if raw_response_text:
    raw_data = []
    lines = raw_response_text.strip().split('\n')
    for line in lines:
        if not line:
            continue
        try:
            raw_data.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Warning: Could not decode line: {line}")

    if raw_data:
        df = pd.DataFrame(raw_data)
        print(f"Successfully processed {len(df)} events.")
    else:
        print("No data was processed.")
else:
    print("No data available to process.")

if len(client_agency) > 0:
    # Filter down to events from a specific state pilot
    mask = df['properties'].apply(lambda p: p.get('client_agency_id') == client_agency)
    df = df[mask]
    
    print(f"Found {len(df)} events with client_agency_id set to", client_agency)

In [ ]:
# Users who shared the income summary
income_events_df = df[df['event'] == 'ApplicantSharedIncomeSummary']
users_who_shared_pdf = income_events_df['properties'].apply(lambda p: p.get('distinct_id')).unique()
print(f"Found {len(users_who_shared_pdf)} users with at least one 'ApplicantSharedIncomeSummary' event.")

# Create a df of just these users
mask = df['properties'].apply(lambda p: p.get('distinct_id')).isin(users_who_shared_pdf)
users_who_shared_pdf_df = df[mask]

# Find all the Finished*Sync events for these users (these are the "mega events")
sync_events = ['ApplicantFinishedArgyleSync', 'ApplicantFinishedPinwheelSync']
sync_events_df = users_who_shared_pdf_df[users_who_shared_pdf_df['event'].isin(sync_events)].copy()

sync_events_df['distinct_id'] = sync_events_df['properties'].apply(lambda p: p.get('distinct_id'))
sync_events_df['w2_count'] = sync_events_df['properties'].apply(lambda p: p.get('employment_type_w2_count', 0))
sync_events_df['gig_count'] = sync_events_df['properties'].apply(lambda p: p.get('employment_type_gig_count', 0))

# Group by the user and calculate the sum of jobs reported
user_employment_counts = sync_events_df.groupby('distinct_id').agg(
    total_w2_count=('w2_count', 'sum'),
    total_gig_count=('gig_count', 'sum')
)

# Df of users who have at least one job
users_with_jobs_df = user_employment_counts[
    ((user_employment_counts['total_w2_count'] + user_employment_counts['total_gig_count'] > 0))
]

print("Total W2 and Gig counts per user: ")
print(user_employment_counts)
print(f"\nFound {len(users_with_jobs_df)} users with at least one W2 or gig source.")
print(users_with_jobs_df.head())

# Df of users who have exactly one job
users_with_one_job_df = user_employment_counts[
    ((user_employment_counts['total_w2_count'] + user_employment_counts['total_gig_count'] == 1))
]

print(f"\nFound {len(users_with_one_job_df)} users with exactly one job.")
print(users_with_one_job_df.head())

In [ ]:
# For debugging, print out the matching events for a single user
specific_user_id = "applicant-99999"
print(f"--- Investigating all sync events for user: {specific_user_id} ---")
single_user_sync_events = sync_events_df[sync_events_df['distinct_id'] == specific_user_id]
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 100, 'display.max_colwidth', 100):
    print(single_user_sync_events[['event', 'properties', 'w2_count', 'gig_count']].T)